# Hybrid model multivariate

We are training a seperate model for each timestep in the lookahead window; multivariate input and output
We are feeding the output of the previous model to each succesive model


## Load functions and data

In [2]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import mean
from numpy import std
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from tensorflow.keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.layers import TimeDistributed
from sklearn.preprocessing import MinMaxScaler
import pywt

In [9]:
def split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead):
    X, y = list(), list()
    example_count = int((len(sequence)/step_interval))
    for i in range(example_count):
        # find the end of this pattern
        end_ix = (i*step_interval) + n_steps_in
        out_start_ix = end_ix + n_step_lookahead -1
        out_end_ix = end_ix + n_steps_out + n_step_lookahead -1
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[(i*step_interval):end_ix], sequence[out_start_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [10]:
#To demonstrate above function
sequence = range(0,13)
n_steps_in = 1
n_steps_in = 5
n_steps_out =1
step_interval =1
n_step_lookahead=5
split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead)

(array([[0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5],
        [2, 3, 4, 5, 6],
        [3, 4, 5, 6, 7]]),
 array([[ 9],
        [10],
        [11],
        [12]]))

In [11]:
percentile_data = pd.read_csv (r'C:\Users/conal/Desktop/MCM/Practicum - Copy/data/block gas price percentile data.csv', header=0)
percentile_data['datetime'] = pd.to_datetime(percentile_data['block_timestamp'], format = '%Y-%m-%d %H:%M:%S UTC')

percentile_data = percentile_data.sort_values(by='datetime',ascending=False)
percentile_data = percentile_data.set_index('datetime')
percentile_data = percentile_data.resample('5T').mean()
percentile_data = percentile_data/1000000000

In [12]:
usage_data = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\ETH,gas,usage merged 11-26 to 05-26.csv', header=0)
usage_data['datetime'] = pd.to_datetime(usage_data['datetime'], format = '%Y-%m-%d %H:%M:%S')
usage_data = usage_data.set_index('datetime')

usage_data = usage_data.squeeze()
usage_data = usage_data.astype('float')
usage_data = usage_data.resample('5T').mean()

In [13]:
usage_data2 = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\Contract counts 2021-11-26 to 2022-05-26.csv', header=0, index_col=0)
usage_data2['datetime'] = pd.to_datetime(usage_data2['block_timestamp'], format = '%Y-%m-%d %H:%M:%S') 
usage_data2 = usage_data2.set_index('datetime')
usage_data2 = usage_data2.drop(['block_timestamp'], axis=1)
usage_data2 = usage_data2.squeeze()
usage_data2 = usage_data2.astype('float')
usage_data2 = usage_data2.resample('5T').sum()

In [14]:
data = usage_data.merge(percentile_data, left_index=True, right_index=True)
data = data.merge(usage_data2, left_index=True, right_index=True)

Load data, datetime to index, downsample with left edge label, convert wei to gwei

In [15]:
def generate_training_val_examples(data):
    #Load data as float, datetime to index





    #Filter inputs
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    #we are only lookign to forecast the min gas price
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))

    
    return X_train, y_train, X_val, y_val, scaler



In [16]:
def LSTM_model():
    model = Sequential()
  
    model.add(LSTM(units =15,activation='tanh',input_shape=(n_steps_in, len(inputs))))
    model.add(Dense(len(inputs)))
    opt = keras.optimizers.Adam(learning_rate=0.01, decay=0.0001)
    model.compile(optimizer='adam', loss='mse')
    return model

In [17]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error


In [18]:
def descale_y_retrun_metrics(yhat, y_val2):
    #We will use validation data that has not had outleirs limited, will be a different min/max scaler as such
    pred_descaled= (scaler.inverse_transform(array([yhat,]*(len(inputs))).transpose()[0]))[:, :1]
    groud_truth_descaled= (scaler2.inverse_transform(array([y_val2,]*(len(inputs))).transpose()[0][0]))[ :, :1]
    RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
    MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
    MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
    MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
    R2 = r2_score(groud_truth_descaled, pred_descaled)
    return RMSE, MAE, MAPE, R2, MSE

## Training loop

# Lets run the loop for one lookahead and one month first

In [19]:
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032
n_steps_out = 1
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
recursive_lookahead=2




dict_indexes=[]
dict_dfs=[]
n_step_lookahead = 1
RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
for month in [0]:
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples(data)
    X_train2, y_train2, X_val2, y_val2, scaler2 = generate_training_val_examples(data)
    RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
    y_hat_list=[]
    train_loss_list=[]
    val_loss_list=[]
    for i in range(1,recursive_lookahead):
        n_step_lookahead = i
        X_train = X_train[:len(y_train)]
        X_val = X_val[:len(y_val2)]



        model = LSTM_model()
        train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1)
        train_loss_list.append(train_history.history['loss'])
        val_loss_list.append(train_history.history['val_loss'])
        model.save(('Hybrid Multivariate/Month' +str(month) +'_' + str(i) + '_step_lookahead'))

        yhat_train = model.predict(X_train, verbose=1)
        yhat_val = model.predict(X_val, verbose=1)
        y_hat_list.append(yhat_val)

        x_train_extended = []
        for k in range(0, len(X_train)):
            X_train_t = np.append(X_train[k], yhat_train[k]).reshape((X_train.shape[1]+1),len(inputs))
            x_train_extended.append(X_train_t)

        x_val_extended = []
        for l in range(0, len(X_val)):
            X_val_t = np.append(X_val[l], yhat_val[l]).reshape((X_val.shape[1]+1),len(inputs))
            x_val_extended.append(X_val_t)

        RMSE, MAE, MAPE, R2, MSE = descale_y_retrun_metrics(yhat_val, y_val2)
        RMSE_list.append(RMSE)
        MAE_list.append(MAE)
        MAPE_list.append(MAPE)
        R2_list.append(R2)
        MSE_list.append(MSE)   

        x_train = np.array(x_train_extended)
        x_val = np.array(x_val_extended)
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],len(inputs))
        x_val = x_val.reshape(x_val.shape[0], x_val.shape[1],len(inputs))
        X_train = x_train[:,1:]
        X_val = x_val[:,1:]



        n_step_lookahead = i+1
        X_traindummy, y_train, X_valdummy, y_val, scaler = generate_training_val_examples(data)
        X_train2, y_train2, X_val2, y_val2, scaler2 = generate_training_val_examples(data)
        if i==max(range(1,recursive_lookahead)):
            pd.DataFrame(train_loss_list).to_csv('Hybrid Multivariate/Month'+str(month)+'_train_loss.csv')
            pd.DataFrame(val_loss_list).to_csv('Hybrid Multivariate/Month'+str(month)+'_val_loss.csv')
            metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=range(1,recursive_lookahead))
            dict_dfs.append(metrics_df)
            dict_indexes.append('Month_' +str(month))
            
        
metrics_dict = dict(zip(dict_indexes, dict_dfs))
np.save("Hybrid Multivariate/metric_dict_1.npy", metrics_dict)       
            

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
108/108 [==============================] - 16s 115ms/step - loss: 0.7137 - val_loss: 0.3590
Epoch 2/15
108/108 [==============================] - 12s 108ms/step - loss: 0.5996 - val_loss: 0.3527
Epoch 3/15
108/108 [==============================] - 12s 107ms/step - loss: 0.5957 - val_loss: 0.3548
Epoch 4/15
108/108 [==============================] - 12s 107ms/step - loss: 0.5948 - val_loss: 0.3516
Epoch 5/15
108/108 [==============================] - 11s 106ms/step - loss: 0.5943 - val_loss: 0.3542
Epoch 6/15
108/108 [==============================] - 12s 107ms/step - loss: 0.5940 - val_loss: 0.3535
Epoch 7/15
108/108 [==============================] - 12s 108ms/step - loss: 0.5939 - val_loss: 0.3511
Epoch 8/15
108/108 [==============================] - 12s 108ms/step - loss: 0.5936 - val_loss: 0.3515
Epoch 9/15
108/108 [==============================] - 11s 106ms/step - loss: 0.5937 - val_loss: 0.3504
Epoch 10/15
108/108 [==============================] - 11s 106ms/step - l

INFO:tensorflow:Assets written to: Hybrid Multivariate/Month0_1_step_lookahead\assets


INFO:tensorflow:Assets written to: Hybrid Multivariate/Month0_1_step_lookahead\assets


46/46 [==============================] - 2s 43ms/step


D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [20]:
metrics_df

,RMSE,MSE,MAE,MAPE,R2
1,24.417865,596.232153,19.611333,0.3192,-0.042374


We are getting a performanace decrease when the model is trained for multiple outputs, further, the model does not seem to learn after a single epoch . This presents an issue for the multivariate hybrid approach; does a different model need to be trained for each variable? This would take apporoximately 6 hours to train for a single model; back testing would take a long time

### Lets get metrics for all variables

In [36]:
RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
for i in range(0,9):
    pred_descaled= (scaler.inverse_transform(array([yhat_val,]*(len(inputs))).transpose()[0]))[:, i]
    groud_truth_descaled= (scaler2.inverse_transform(array([y_val,]*(len(inputs))).transpose()[0][0]))[ :, i]
    

    
    RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
    MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
    MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
    MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
    R2 = r2_score(groud_truth_descaled, pred_descaled)
    RMSE_list.append(RMSE)
    MAE_list.append(MAE)
    MAPE_list.append(MAPE)
    R2_list.append(R2)
    MSE_list.append(MSE) 
    
metrics = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=inputs)

    

In [40]:
metrics

,RMSE,MSE,MAE,MAPE,R2
min_gas_price,24.417865,5.962322e+02,19.611333,0.319200,-0.042374
block_gas_5th_percentile,26.712095,7.135360e+02,21.453955,0.363973,-0.042374
block_gas_95th_percentile,42.668984,1.820642e+03,34.269812,0.429470,-0.042374
gas_used,669550.810325,4.482983e+11,537753.146371,0.032236,-0.042374
base_fee_per_gas,21.093605,4.449402e+02,16.941436,0.264557,-0.042374
transaction_count,12.819530,1.643404e+02,10.296071,0.051783,-0.042374
size,6869.776577,4.719383e+07,5517.496045,0.060487,-0.042374
Open,186.300499,3.470788e+04,149.628193,0.047305,-0.042374
contracts,17.014927,2.895077e+02,13.665625,0.357373,-0.042374
